In [ ]:

import os
from pathlib import Path

import numpy as np
import pandas as pd
from Bio import SeqIO
import torch
import esm

# Reproducibility
random_seed = 42
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
np.random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Hardware + file system configuration
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

PROJECT_ROOT = Path('/content')
print(f"Using device: {device}")
POS_FASTA = PROJECT_ROOT / 'reps_30_rep_seq_pos.fasta'
NEG_FASTA = PROJECT_ROOT / 'reps_30_rep_seq_neg.fasta'
OUTPUT_DIR = PROJECT_ROOT / 'esm_outputs'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MAX_SEQUENCES_PER_BATCH = 8
MAX_TOKENS_PER_BATCH = 2000  # approximate per-batch token budget
MAX_RESIDUES_PER_CHUNK = 1000  # split very long sequences
MAX_TOKENS_PER_BATCH = 2000  # approximate per-batch token budget

ESM_MODEL = 'esm2_t6_8M_UR50D'
META_KEYS = ['source', 'gene', 'entry', 'file', 'idx']

for fasta_path in [POS_FASTA, NEG_FASTA]:
    if not fasta_path.exists():
        raise FileNotFoundError(f"Missing FASTA file: {fasta_path}")


In [ ]:

VALID_RESIDUES = set("ACDEFGHIKLMNPQRSTVWYBJOUXZ")

def is_valid_sequence(seq: str) -> bool:
    return set(seq.upper()) <= VALID_RESIDUES


def parse_header_metadata(header: str) -> dict:
    parts = header.split('|')
    meta = {'source': parts[0]}
    for part in parts[1:]:
        if '=' in part:
            key, value = part.split('=', 1)
            meta[key.strip()] = value.strip()
    for key in META_KEYS:
        meta.setdefault(key, 'NA')
    return meta


def read_fasta_with_metadata(file_path: Path, label: int) -> pd.DataFrame:
    records = []
    for record in SeqIO.parse(str(file_path), 'fasta'):
        meta = parse_header_metadata(record.description)
        records.append({
            **meta,
            'header': record.description,
            'sequence': str(record.seq),
            'label': label
        })
    return pd.DataFrame(records)


positive_df = read_fasta_with_metadata(POS_FASTA, 1)
negative_df = read_fasta_with_metadata(NEG_FASTA, 0)
print(f"Loaded {len(positive_df)} positive and {len(negative_df)} negative representatives")

# Combine, shuffle (for downstream ML), and index sequences
combined_df = pd.concat([positive_df, negative_df], ignore_index=True)
valid_mask = combined_df['sequence'].str.upper().apply(lambda s: set(s) <= VALID_RESIDUES)
invalid = combined_df[~valid_mask]
if len(invalid):
    print(f'Removed {len(invalid)} sequences with non-standard residues')
combined_df = combined_df[valid_mask].reset_index(drop=True)
sequence_df = combined_df.sample(frac=1.0, random_state=random_seed).reset_index(drop=True)
sequence_df.insert(0, 'sequence_id', range(len(sequence_df)))
sequence_df['sequence_length'] = sequence_df['sequence'].str.len()

metadata_columns = ['sequence_id', 'label'] + META_KEYS + ['header', 'sequence_length']
metadata_df = sequence_df[metadata_columns].copy()
print(f"Unique genes represented: {metadata_df['gene'].nunique()}")
print(sequence_df.groupby('label')['sequence_length'].describe()[['mean', 'min', 'max']])


In [ ]:

model_loader = getattr(esm.pretrained, ESM_MODEL)
model, alphabet = model_loader()
model = model.eval().to(device)
repr_layer = model.num_layers
batch_converter = alphabet.get_batch_converter()


def chunked_rows(df: pd.DataFrame):
    for row in df.itertuples():
        sequence = row.sequence
        if len(sequence) <= MAX_RESIDUES_PER_CHUNK:
            yield row, sequence
        else:
            for start in range(0, len(sequence), MAX_RESIDUES_PER_CHUNK):
                yield row, sequence[start:start + MAX_RESIDUES_PER_CHUNK]


def batch_iter(df: pd.DataFrame):
    batch_rows = []
    token_budget = 0
    for row, chunk_seq in chunked_rows(df):
        length = len(chunk_seq) + 2  # account for BOS/EOS tokens
        if batch_rows and (
            len(batch_rows) >= MAX_SEQUENCES_PER_BATCH or
            token_budget + length > MAX_TOKENS_PER_BATCH
        ):
            yield batch_rows
            batch_rows = []
            token_budget = 0
        batch_rows.append((row, chunk_seq))
        token_budget += length
    if batch_rows:
        yield batch_rows


def embed_dataframe(df: pd.DataFrame) -> np.ndarray:
    embeddings = np.zeros((len(df), model.embed_dim), dtype=np.float32)
    chunk_counts = np.zeros(len(df), dtype=np.int32)
    processed = 0
    for batch_rows in batch_iter(df):
        batch_ids = [str(row.sequence_id) for row, _ in batch_rows]
        batch_seqs = [seq for _, seq in batch_rows]
        batch_data = list(zip(batch_ids, batch_seqs))
        _, _, batch_tokens = batch_converter(batch_data)
        batch_tokens = batch_tokens.to(device)
        with torch.no_grad():
            results = model(batch_tokens, repr_layers=[repr_layer], return_contacts=False)
        token_representations = results['representations'][repr_layer]
        seq_representations = token_representations[:, 1:-1].mean(1).cpu().numpy()
        for (row, _), embedding in zip(batch_rows, seq_representations):
            embeddings[row.Index] += embedding
            chunk_counts[row.Index] += 1
        processed += len(batch_rows)
        if processed % 200 == 0:
            print(f"Processed {processed} chunks")
    if not np.all(chunk_counts):
        missing = np.where(chunk_counts == 0)[0]
        raise RuntimeError(f"Missing embeddings for rows: {missing[:10]} ...")
    embeddings /= chunk_counts[:, None]
    return embeddings


embeddings = embed_dataframe(sequence_df)
labels = sequence_df['label'].to_numpy(dtype=np.int8)
np.save(OUTPUT_DIR / 'esm_features.npy', embeddings)
np.save(OUTPUT_DIR / 'labels.npy', labels)
metadata_df.to_csv(OUTPUT_DIR / 'sequence_metadata.csv', index=False)
metadata_df.to_parquet(OUTPUT_DIR / 'sequence_metadata.parquet', index=False)

print('Saved embeddings to', OUTPUT_DIR / 'esm_features.npy')
print('Saved labels to', OUTPUT_DIR / 'labels.npy')
print('Saved metadata to', OUTPUT_DIR / 'sequence_metadata.(csv|parquet)')


In [ ]:

import numpy as np


def summarize_gene_performance(predictions, metadata: pd.DataFrame, threshold: float = 0.5) -> pd.DataFrame:
    """Link downstream model predictions back to genes for diagnostics."""
    scores = np.asarray(predictions)
    if len(scores) != len(metadata):
        raise ValueError('Predictions and metadata must have the same length')

    if scores.ndim == 2:
        if scores.shape[1] == 2:
            scores = scores[:, 1]
        elif scores.shape[1] == 1:
            scores = scores[:, 0]
        else:
            raise ValueError('Unsupported prediction shape; expected (n,), (n,1) or (n,2)')

    df = metadata.copy().reset_index(drop=True)
    df['prediction_score'] = scores
    df['pred_label'] = (df['prediction_score'] >= threshold).astype(int)
    df['correct'] = (df['pred_label'] == df['label'])

    summary = (
        df.groupby(['gene', 'label'])
          .agg(
              n_sequences=('sequence_id', 'count'),
              accuracy=('correct', 'mean')
          )
          .reset_index()
          .sort_values('accuracy', ascending=False)
    )
    return summary


# Example usage after training a classifier:
# preds = trained_model.predict_proba(embeddings)[:, 1]
# gene_perf = summarize_gene_performance(preds, metadata_df)
# gene_perf.head()
